# Building an Anime Recommender System

In [1]:
from animerec import datasets, utils
import pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### 1. Load Dataset and Model

In [2]:
import sys
sys.modules['datasets'] = datasets
sys.modules['utils'] = utils

# Load dataset for cindex, id_to_anime, etc.
pickle_in = open("data/MyAnimeList.pickle", 'rb')
data = pickle.load(pickle_in)
pickle_in.close()

# Load model trained on server.
pickle_in = open("animerec/model/model.pickle", 'rb')
model = pickle.load(pickle_in)
pickle_in.close()

### 3. Print Features in Model

In [3]:
utils.print_features(model, data, nfeatures=7)

Printing 7 of 40 features.
Feature 1:
	max : (+4.0) Suzumiya Haruhi no Yuuutsu
	min : (-6.0) Naruto: Shippuuden
Feature 2:
	max : (+5.0) .hack//Sign
	min : (-3.1) Tsubasa Chronicle 2nd Season
Feature 3:
	max : (+4.7) Sword Art Online
	min : (-3.3) Higashi no Eden Movie II: Paradise Lost
Feature 4:
	max : (+7.0) Pingu in the City
	min : (-4.1) Lucky☆Star
Feature 5:
	max : (+4.5) Bobobo-bo Bo-bobo
	min : (-4.4) Aki-Sora
Feature 6:
	max : (+5.9) Ore no Imouto ga Konnani Kawaii Wake ga Nai. Specials
	min : (-5.2) School Days
Feature 7:
	max : (+3.6) Boku no Pico
	min : (-7.6) School Days


### 4. Webscrape current profile and create prediction

In [4]:
myprofile = utils.get_user_anime_list('Manuel')
x = utils.get_score_vector_from_user_anime_list(myprofile, data.cindex)
xhat = model.predict(x)
prediction = utils.prediction_to_dataframe(xhat, myprofile, data.cindex, keep_all=False)

print('Top 10 Anime Predictions')
for i in prediction.sort_values(ascending=False).head(10).index:
    str_ = (
        f"\t({prediction[i]:2.4f}) {data.get_anime_by_id(i).title}"
    )
    print(str_)
    
print('\nBottom 10 Anime Predictions')
for i in prediction.sort_values(ascending=False).tail(10).index:
    str_ = (
        f"\t({prediction[i]:2.4f}) {data.get_anime_by_id(i).title}"
    )
    print(str_)

Scraped 300 additional anime from https://myanimelist.net/animelist/Manuel/load.json?offset=0&status=7
Scraped 156 additional anime from https://myanimelist.net/animelist/Manuel/load.json?offset=300&status=7
Top 10 Anime Predictions
	(10.9228) Nana
	(10.6595) Kenpuu Denki Berserk
	(9.3060) Paradise Kiss
	(9.2003) Devilman: Crybaby
	(9.1187) Ginga Eiyuu Densetsu
	(8.8565) Nodame Cantabile Finale
	(8.8526) Koe no Katachi
	(8.7426) Yoru wa Mijikashi Arukeyo Otome
	(8.6877) Nekojiru-sou
	(8.6784) Kono Sekai no Katasumi ni

Bottom 10 Anime Predictions
	(0.9226) Final Fantasy: Unlimited
	(0.8534) Makura no Danshi
	(0.7679) Tenshi Kinryouku
	(0.7201) Angel&#039;s Feather
	(0.6987) Suki na Mono wa Suki Dakara Shou ga Nai!!
	(0.5072) 07-Ghost
	(0.1106) Zombie-Loan
	(-0.0059) Junjou Romantica 2
	(-0.7025) Junjou Romantica
	(-0.7065) Gakuen Heaven
